## preprocess

In [1]:
import numpy as np

from preprocess import run_prepro

train_sents, train_labels, valid_sents, valid_labels, test_sents, test_labels, word_map = \
    run_prepro("./data/restaurant", ".", 200)  # load all data


Training data: reading and preprocessing...



  0%|          | 0/105000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\13080\AppData\Local\Temp\jieba.cache
Loading model cost 0.363 seconds.
Prefix dict has been built successfully.
100%|██████████| 105000/105000 [01:59<00:00, 881.68it/s]



Training data: discarding words with counts less than 10, the size of the vocabulary is 35230.

Training data: word map saved to C:\Users\13080\Desktop\SCIR-SC-get-started\final-proj.

Valid data: reading and preprocessing...



100%|██████████| 15000/15000 [00:17<00:00, 839.60it/s]


Test data: reading and preprocessing...



100%|██████████| 15000/15000 [00:17<00:00, 866.34it/s]

All done!



In [3]:
print(train_labels[:5])
print(train_sents[:5])
# print(word_map)

[1, -2, 0, -2, -2]
[['吼吼', '吼', '，', '萌死', '人', '的', '棒棒糖', '，', '中', '了', '大众', '点评', '的', '霸王餐', '，', '太', '可爱', '了', '。', '一直', '就', '好奇', '这个', '棒棒糖', '是', '怎么', '个', '东西', '，', '大众', '点评', '给', '了', '我', '这个', '土老冒', '一个', '见识', '的', '机会', '。', '看', '介绍', '棒棒糖', '是', '用', '德国', '糖', '做', '的', '，', '不会', '很甜', '，', '中间', '的', '照片', '是', '糯米', '的', '，', '能', '食用', '，', '真是太', '高端', '大气', '上档次', '了', '，', '还', '可以', '买', '蝴蝶结', '扎口', '，', '送人', '可以', '买', '礼盒', '。', '我', '是', '先', '打', '的', '卖家', '电话', '，', '加', '了', '微信', '，', '给', '卖家', '传', '的', '照片', '。', '等', '了', '几天', '，', '卖家', '就', '告诉', '我', '可以', '取货', '了', '，', '去', '大官', '屯', '那取', '的', '。', '虽然', '连', '卖家', '的', '面', '都', '没', '见到', '，', '但是', '还是', '谢谢', '卖家', '送', '我', '这么', '可爱', '的', '东西', '，', '太', '喜欢', '了', '，', '这', '哪', '舍得吃', '啊'], ['第三次', '参加', '大众', '点评', '网', '霸王餐', '的', '活动', '。', '这家', '店', '给', '人', '整体', '感觉', '一般', '。', '首先', '环境', '只能', '算', '中等', '，', '其次', '霸王餐', '提供', '的', '菜品', '也', '不是', '很多', '，

In [8]:
import numpy as np

# print(valid_sents[:5])
print(test_sents[:1])
print(test_labels[:3])

[['我', '想', '说', '他们', '家', '的', '优惠活动', '好', '持久', '啊', '，', '我', '预售', '的', '时候', '买', '的', '券', '，', '前两天', '心血来潮', '去', '吃', '的', '活动', '还', '在', '继续', '首先', '说', '下', '服务', '，', '因为', '和', '男票', '开车', '去', '的', '，', '有点', '不', '认路', '，', '老板', '很', '耐心', '的', '在', '电话', '里', '帮', '我们', '指路', '，', '到', '了', '门店', '之后', '也', '帮', '我们', '推荐', '了', '他们', '家', '做', '的', '比较', '地道', '的', '伤心', '凉粉', '，', '说', '是', '厨师', '是', '四川', '那边', '来', '的', '。', '环境', '呢', '比较简单', '干净', '，', '去', '的', '时候', '下午', '一点多', '了', '，', '还有', '四五桌', '人', '在', '用餐', '口味', '对于', '我', '而言', '点', '了', '麻辣', '的', '口感', '正', '正好', '，', '男票', '比较', '能', '吃', '辣', '，', '相对而言', '觉得', '他们', '家', '的', '麻辣', '口感', '麻有', '了', '，', '辣', '还', '欠缺', '一点', '，', '老板娘', '说', '考虑', '到', '客人', '口味', '不同', '所以', '没敢', '放太多', '辣椒', '，', '能', '吃', '辣', '的', '朋友', '可以', '考虑', '下单', '之前', '和', '老板', '先', '说好', '。', '鱼', '呢', '我们', '选', '的', '是', '黑鱼', '，', '2.9', '斤', '的', '鱼', '加上', '一盆', '我', '以为', '没有', '什么', '东西', '实际上', '东西'

## record hyper-params and import dependency

In [10]:
from argparse import Namespace

config = Namespace(
    lr = 0.001,
    epoch = 5,
    word_emb_dim = 300,
    hidden_dim = 256,
    batch_size = 64
)

from tqdm import tqdm
from torch.utils.data import DataLoader
from models import LSTM
from datasets import ReviewDataset
import torch.nn as nn
from torch import optim
import torch
import torch.backends.cudnn as cudnn
from tutils import valid_and_save

cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print("current device on :")
print(device)

current device on :
cuda


## main

In [36]:


train_set = ReviewDataset(train_sents, train_labels, word_map, 200)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(ReviewDataset(valid_sents, valid_labels, word_map, 200), batch_size=64, shuffle=False, pin_memory=True)

model = LSTM(300, 4, len(word_map), 256)
model.train()


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    params=filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.001
)
epoch_num = 5
ckpt_path = "./ckpt"


model = model.to(device)
loss_fn = loss_fn.to(device)

use_wandb = False
# log setting
if use_wandb:
    import wandb
    proj = "fine-grained-SA"
    wandb.init(project=proj, config=config)



for epoch in range(epoch_num):
    for i, batch in tqdm(enumerate(train_loader)):
        sents, labels, len_per_sent = batch
        sents = sents.to(device)
        labels = labels.squeeze(-1).to(device)   # B
        # print("labels:")
        # print(labels)
        logits = model(sents, len_per_sent.squeeze(-1).to(device))
        # print("logits:")
        # print(logits)
        loss = loss_fn(logits, labels)
        # print("loss")
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        # grad norm clipping prevents gradient exploding
        gnorm = nn.utils.clip_grad_norm_(model.parameters(), 1.0)


        optimizer.step()

        # calculate the accuracy
        pred_labels = torch.max(logits, dim=1)[1]
        # print(pred_labels)
        acc = torch.eq(pred_labels, labels).sum().item() / labels.shape[0]

        # print log per 1000steps
        if i % 1000 == 0:
            print(
                'Epoch: [{0}][{1}/{2}]\t'
                'Loss_avg: {loss:.4f}\t'
                'Accuracy {acc:.3f} '.format(
                    epoch, i, len(train_loader),
                    loss = loss,
                    acc = acc
                )
            )
        if use_wandb:
            wandb.log({
                        "train/loss_avg":loss,
                        "train/acc": acc,
                        "train/gnorm": gnorm.item()
                       })
    # 一个epoch，在valid上做验证
    total_avg_loss, total_correct = valid_and_save(valid_dataloader, device, model, loss_fn, epoch, ckpt_path)
    print("Loaded one epoch. Validate finished.")
    print(f"Avg_loss on valid_set:{total_avg_loss / len(valid_dataloader): .4f}\n"
          f"Avg_acc on valid_set:{total_correct / len(valid_labels): .4f}")

    if use_wandb:
        wandb.log({
            "valid/loss": total_avg_loss / len(valid_dataloader),
            "valid/acc": total_correct / len(valid_labels)
        }, commit=False)


1it [00:00,  1.31it/s]

Epoch: [0][0/1641]	Loss_avg: 1.3963	Accuracy 0.219 


5it [00:04,  1.02it/s]


KeyboardInterrupt: 

In [8]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 2050


In [ ]:
total_correct / len(valid_labels)

## model weight save

In [11]:
# torch.save(model, "./epoch2-ckpt.pth")
model2 = torch.load("./epoch2-ckpt.pth")
print(model2.state_dict())

OrderedDict([('embeddings.weight', tensor([[ 0.7266, -0.2000, -1.4011,  ..., -0.1781,  2.4834,  0.7634],
        [-0.1298, -1.6156, -0.4743,  ..., -0.4352, -0.9377,  0.6959],
        [-0.2147,  0.8819,  1.3596,  ...,  0.2694,  0.3832, -1.0314],
        ...,
        [-1.2936, -0.6976, -0.4473,  ..., -1.5189, -1.9226,  0.4665],
        [-0.1499, -2.0439,  1.5591,  ...,  1.7281, -1.0863, -0.6732],
        [ 2.4724,  0.9626,  0.1072,  ..., -0.4259, -1.5169, -0.5108]],
       device='cuda:0')), ('rnn.weight_ih_l0', tensor([[-0.1169,  0.0185, -0.1637,  ...,  0.1324,  0.0317, -0.1420],
        [-0.0653, -0.1709, -0.1360,  ...,  0.0028,  0.0201, -0.0251],
        [-0.0672, -0.0042,  0.0041,  ...,  0.0009,  0.0261,  0.0422],
        ...,
        [ 0.0413,  0.1204,  0.0683,  ...,  0.0821,  0.0372, -0.1424],
        [-0.0985,  0.0340,  0.0923,  ...,  0.0496, -0.0896,  0.0790],
        [-0.0563, -0.0509,  0.0051,  ...,  0.0089,  0.0553,  0.0749]],
       device='cuda:0')), ('rnn.weight_hh_l0', ten

## validate and save

In [13]:


loss_fn = nn.CrossEntropyLoss()
model2 = model2.to(device)
loss_fn = loss_fn.to(device)
model2.eval()

print(model2.state_dict())
print(loss_fn)

valid_dataloader = DataLoader(ReviewDataset(valid_sents, valid_labels, word_map, 200), batch_size=64, shuffle=False, pin_memory=True)
total_loss = 0
total_correct = 0

with torch.no_grad():
    for i, batch in tqdm(enumerate(valid_dataloader)):
        sents, labels, len_per_sent = batch
        sents = sents.to(device)
        labels = labels.squeeze(-1).to(device)   # B
        # print("labels:")
        # print(labels)
        logits = model2(sents, len_per_sent.squeeze(-1).to(device))
        # print("logits:")
        # print(logits)
        loss = loss_fn(logits, labels)
        # print("loss")
        # print(loss)
        # calculate the accuracy
        pred_labels = torch.max(logits, dim=1)[1]
        # print(pred_labels)
        correct = torch.eq(pred_labels, labels).sum().item()
        total_loss += loss.item()
        total_correct += correct
print(f"valide_epoch_avg_loss:{total_loss / len(valid_dataloader) : .4f}")
print(f"valide_epoch_avg_acc:{total_correct / len(valid_labels): .4f}")


current device on :
cuda
OrderedDict([('embeddings.weight', tensor([[ 0.7266, -0.2000, -1.4011,  ..., -0.1781,  2.4834,  0.7634],
        [-0.1298, -1.6156, -0.4743,  ..., -0.4352, -0.9377,  0.6959],
        [-0.2147,  0.8819,  1.3596,  ...,  0.2694,  0.3832, -1.0314],
        ...,
        [-1.2936, -0.6976, -0.4473,  ..., -1.5189, -1.9226,  0.4665],
        [-0.1499, -2.0439,  1.5591,  ...,  1.7281, -1.0863, -0.6732],
        [ 2.4724,  0.9626,  0.1072,  ..., -0.4259, -1.5169, -0.5108]],
       device='cuda:0')), ('rnn.weight_ih_l0', tensor([[-0.1169,  0.0185, -0.1637,  ...,  0.1324,  0.0317, -0.1420],
        [-0.0653, -0.1709, -0.1360,  ...,  0.0028,  0.0201, -0.0251],
        [-0.0672, -0.0042,  0.0041,  ...,  0.0009,  0.0261,  0.0422],
        ...,
        [ 0.0413,  0.1204,  0.0683,  ...,  0.0821,  0.0372, -0.1424],
        [-0.0985,  0.0340,  0.0923,  ...,  0.0496, -0.0896,  0.0790],
        [-0.0563, -0.0509,  0.0051,  ...,  0.0089,  0.0553,  0.0749]],
       device='cuda:0')),

235it [00:03, 69.49it/s]

valide_epoch_avg_loss: 0.7804
valide_epoch_avg_acc: 0.7151


## Test

In [24]:

test_set = ReviewDataset(test_sents, test_labels, word_map, 200)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=False, pin_memory=True)

# load best model
model = torch.load("./ckpt/epoch2-09-12-2022-21-05-13-ckpt.pth")
model = model.to(device)

labels = []

for i, batch in tqdm(enumerate(test_dataloader)):
    sents, _, len_per_sent= batch
    logits = model(sents.to(device), len_per_sent.squeeze(-1).to(device)) # B x C
    pred_labels = torch.max(logits, dim=1)[1] # B
    labels += (pred_labels - 2).tolist()

print(labels)

235it [00:43,  5.35it/s]

[1, -2, -1, 1, 1, 1, 1, -2, -2, -2, -2, 1, -2, -2, 1, -2, 1, -2, 1, -2, 0, 1, -1, 0, -2, -2, -2, 1, -2, -2, 1, 1, -1, -2, -2, -2, 1, -2, 1, -2, -1, 1, -2, -2, 1, -2, 1, 1, 1, -2, 0, -2, -2, 1, -2, -2, 1, 1, 1, 1, 1, 1, 1, -2, 1, 1, -2, -2, -2, 1, -2, 1, -2, 1, 1, -2, -2, 1, -2, -2, 1, -2, -2, -2, -1, 1, 1, -2, -2, 1, 1, -2, 1, -1, 1, -2, -2, -2, 0, -2, -2, 1, 1, -2, 0, -2, 1, 0, -2, -2, -2, 1, 1, 1, -2, 1, 1, 0, -2, -2, 1, -2, -1, 1, 1, -2, 1, 1, -2, -2, -2, 0, 1, 1, -2, 0, 1, 0, -2, 1, 1, 1, 1, 1, 0, 0, 0, 1, -2, 0, 1, -2, -2, -2, -2, 1, -2, 1, -2, -2, 1, -2, 0, -1, 1, -2, -2, -2, -2, -2, 1, -2, 1, -2, -1, -2, -2, 1, -1, -2, -2, -2, -1, 0, -1, -2, -2, 1, 1, -1, -2, -2, 1, 1, -2, -2, -1, -2, 0, -2, -2, -2, -2, -2, 0, 1, 1, -2, 1, -2, 1, 1, 0, -2, 1, 1, 1, -2, -2, -2, -2, 1, 1, 1, 0, -2, -2, -2, -2, -2, 1, -2, 1, -2, -2, 1, 1, -1, -2, -2, -2, -2, 1, -2, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, -2, 1, -2, 1, 1, 1, 1, -2, -2, -2, -2, -2, -1, -2, 1, 1, -2, -2, 1, -2, -2, 1, -2, -1, -2, -2, -1, -2, -2

## append to test.csv

In [33]:
import pandas as pd
df = pd.DataFrame(labels)
print(df)
df.to_csv("./data/restaurant/test2.csv", index=False, header=["waiters_attitude"])

       0
0      1
1     -2
2     -1
3      1
4      1
...   ..
14995  1
14996  1
14997  1
14998  1
14999 -2

[15000 rows x 1 columns]


## inference demo   --- lay it asaide for now

In [13]:
a = [1,2,3]
print(a - 1)

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [69]:
a = torch.LongTensor([[145],
        [139]])
print(a)
print(a.squeeze(-1))

tensor([[145],
        [139]])
tensor([145, 139])
